# Init

In [1]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
from psycopg2.extras import NamedTupleCursor
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [2]:
from data_manager.database_manager import db_fetchone,db_no_fetch,db_fetchall
from core.utils.log import log_info,log_error
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        log_info('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchone().version
                log_info(f"PostgreSQL database version: {db_version}")
                log_info(f"PostgreSQL connection status: {conn.info.status}")
                log_info(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        log_error(error)
        conn = None


In [3]:
conn=init_connection(dsn)

2021-07-25 10:36:52.808 INFO    root: Connecting to the PostgreSQL database...
2021-07-25 10:36:52.839 INFO    root: PostgreSQL database version: PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit
2021-07-25 10:36:52.842 INFO    root: PostgreSQL connection status: 0
2021-07-25 10:36:52.844 INFO    root: You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'.


# 23/7/2021

In [17]:
from enum import Enum,IntEnum
from typing import NamedTuple

In [18]:
class Detection(NamedTuple):
    name: str
    prob: float

In [19]:
detection=Detection('car',0.95)

In [35]:
detection.name,detection.prob
detection

Detection(name='car', prob=0.95)

In [37]:
import numpy as np
x=np.array([1,2,3])

In [39]:
np.asarray(x)

array([1, 2, 3])

# 24/7/2021

In [13]:
query_project_SQL="""
                    SELECT
                        p.id,
                            p.name,
                            description,
                            deployment_id,
                            project_path,
                            dt.name as deployment_type
                    FROM
                        public.project p
                        LEFT JOIN deployment_type dt ON dt.id = p.deployment_id
                    WHERE
                        p.id = 2;"""

In [15]:
db_fetchone(query_project_SQL,conn)

Record(id=2, name='First Project', description='This project is for object detection', deployment_id=2, project_path='/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/project', deployment_type='Object Detection with Bounding Boxes')

### Get column names for DB

In [23]:

data_rows=[]
with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute("""select pt.id as "ID", pt.name as "Name",f.name as "Framework",model_path as "Model Path", pt.updated_at as "Date/Time" from pre_trained_models pt left join framework f on framework_id = f.id;""")
                column_names = [desc[0] for desc in cur.description]
                for row in cur:
                    data_rows.append(row)
print("Column names: {}\n".format(column_names))


Column names: ['ID', 'Name', 'Framework', 'Model Path', 'Date/Time']



In [14]:
from psycopg2.extras import NamedTupleCursor
with conn.cursor(cursor_factory=NamedTupleCursor) as cur:
    cur.execute("""
                select pt.id as "ID", pt.name as "Name",f.name as "Framework",model_path as "Model Path", pt.updated_at as "Date/Time" from pre_trained_models pt left join framework f on framework_id = f.id;
                    """)
    return_all=cur.fetchall()

In [14]:
SQL_msg="""
                select pt.id as "ID", pt.name as "Name",f.name as "Framework",model_path as "Model Path", pt.updated_at as "Date/Time" from pre_trained_models pt left join framework f on framework_id = f.id;
                    """
return_all,column_names=db_fetchall(SQL_msg,conn=conn,fetch_col_name=True)

In [5]:
for i in return_all:
    display(i)

Record(ID=1, Name='[TF] SSD MobileNet V2 FPNLite 320x320', Framework='TensorFlow', Model_Path='./pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8', Date_Time=datetime.datetime(2021, 7, 24, 23, 55, 39, 792579, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)))

Record(ID=2, Name='[TF] SSD ResNet50 V1 FPN 640x640 (RetinaNet50)', Framework='TensorFlow', Model_Path='./pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8', Date_Time=datetime.datetime(2021, 7, 24, 23, 55, 39, 792579, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)))

In [22]:
column_names=None

In [9]:
import pandas as pd

####  Create Pandas Dataframe using PostgreSQL column names

In [11]:
df=pd.DataFrame(return_all,columns=column_names)
display(df)

,ID,Name,Framework,Model_Path,Date_Time
0,1,[TF] SSD MobileNet V2 FPNLite 320x320,TensorFlow,./pre-trained-models/ssd_mobilenet_v2_fpnlite_...,2021-07-24 23:55:39.792579+08:00
1,2,[TF] SSD ResNet50 V1 FPN 640x640 (RetinaNet50),TensorFlow,./pre-trained-models/ssd_resnet50_v1_fpn_640x6...,2021-07-24 23:55:39.792579+08:00


In [56]:
from pathlib import Path
x=Path.home()
x

PosixPath('/home/rchuzh')

In [67]:
y=x/'./programming'

In [68]:
y

PosixPath('/programming')

# 25/7/2021

In [6]:
from core.utils.helper import create_dataframe

In [15]:
create_dataframe(return_all,column_names=column_names)

NameError: name 'column_names' is not defined